In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

In [ ]:
df_ASPM = pd.concat(
    [pd.read_csv(ZipFile('ASPM.zip').open(i)) for i in ZipFile('ASPM.zip').namelist()],
    ignore_index=True
)
df_ASPM['Facility'] = df_ASPM['Facility'].str.strip()

df_ASPM.head()

In [ ]:
weather_zip = ZipFile('AirportWeather.zip')

df_weather = pd.concat(
    [pd.read_csv(weather_zip.open(i), comment="#", skiprows=[11]) for i in weather_zip.namelist()],
    ignore_index=True
)

df_weather.head()

In [ ]:

# for i in range(len(df_ASPM.index)):
#     num_departures = df_ASPM.loc[i, 'DeparturesFor MetricComputation']
#     percent_ontime = df_ASPM.loc[i, '% On-TimeAirportDepartures']
#     ontime_departures = round(num_departures * (percent_ontime/100))
#     for j in range(num_departures):
#         delayed = not (j < ontime_departures)
#         departures.append(df_ASPM.loc[i, ['Hour', 'Date', 'Facility']])

departures = np.empty((0, 4))

for row in df_ASPM[df_ASPM['Facility'] == 'LAX'].itertuples(index=False):
   num_departures = row[6]
   percent_ontime = row[9]
   ontime_departures = round(num_departures * (percent_ontime/100))
   np_row = np.array([row[0], row[2], row[3]])
   ontime = np.full((ontime_departures, 4), np.append(np_row, False))
   delayed = np.full((num_departures - ontime_departures, 4), np.append(np_row, True))
   departures = np.concatenate((departures, ontime, delayed))